# Orchestration Multiple LLMs

In [19]:
# Import Required Libraries
import os
from dotenv import load_dotenv 

In [20]:
load_dotenv(override=True)

True

In [21]:
import openai

openai_api_key = os.getenv("OPENAI_API_KEY")
gemini_api_key = os.getenv("GEMINI_API_KEY")

# Ensure the API keys are loaded correctly
if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable not set. Please set it in your .env file.")
else:
    openai.api_key = openai_api_key

if gemini_api_key is None:
    raise ValueError("GEMINI_API_KEY environment variable not set. Please set it in your .env file.")
else:
    # Return both API keys
    (openai_api_key, gemini_api_key)

In [22]:
message = [{"role": "system", "content": "You are a helpful assistant."}]
# Function to get a response from the OpenAI API
def get_response(prompt):
    message.append({"role": "user", "content": prompt})
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=message,
        temperature=0.7,
        max_tokens=150
    )
    message.append(response.choices[0].message)
    return response.choices[0].message.content

In [23]:
question = "What is the capital of France?"
response = get_response(question)
print(f"Question: {question}")

Question: What is the capital of France?


In [ ]:
competitors = []
answer = [] 
def prepare_messages(question):
    return [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": question}]


In [25]:
def get_competitor_response(model_name, messages):
    response = openai.chat.completions.create(
        model=model_name,
        messages=messages,
        temperature=0.7,
        max_tokens=150
    )
    competitors.append(response.choices[0].message.content)
    answer.append(response.choices[0].message.content)
    print(f"Competitor Response: {competitors[-1]}")



In [26]:
model_name = "gpt-4" 
# Call the function
messages = prepare_messages(question)
# Get the competitor's response
get_competitor_response(model_name, messages)

Competitor Response: The capital of France is Paris.


In [27]:
from openai import OpenAI

def get_competitor_response_gemini(model_name, messages):
    # Initialize the Gemini API client
    client = OpenAI(
        api_key=gemini_api_key,
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    
    # Get the response from the Gemini API
    response = client.chat.completions.create(
        model=model_name,
        messages=messages
    )
    
    # Extract and store the response content
    competitors.append(response.choices[0].message.content)
    answer.append(response.choices[0].message.content)
    print(f"Competitor Response (Gemini): {competitors[-1]}")

In [28]:
model_name = "gemini-2.0-flash" 
# Call the function
messages = prepare_messages(question)
# Get the competitor's response
get_competitor_response_gemini(model_name, messages)


Competitor Response (Gemini): The capital of France is **Paris**.



In [29]:
def interactive_conversation(model1_name, model2_name, initial_message, num_turns=5):
    # Prepare the initial message
    messages_model1 = prepare_messages(initial_message)
    messages_model2 = prepare_messages(initial_message)
    
    print(f"Initial Message: {initial_message}")
    
    for turn in range(num_turns):
        # Get response from model 1
        print(f"\nTurn {turn + 1}:")
        print(f"Model 1 ({model1_name}) responding...")
        get_competitor_response(model1_name, messages_model1)
        model1_response = competitors[-1]
        print(f"Model 1 Response: {model1_response}")
        
        # Add model 1's response to model 2's messages
        messages_model2.append({"role": "user", "content": model1_response})
        
        # Get response from model 2
        print(f"Model 2 ({model2_name}) responding...")
        get_competitor_response_gemini(model2_name, messages_model2)
        model2_response = competitors[-1]
        print(f"Model 2 Response: {model2_response}")
        
        # Add model 2's response to model 1's messages
        messages_model1.append({"role": "user", "content": model2_response})

In [30]:
from pprint import pprint
from IPython.display import Markdown, display

# Start an interactive conversation
# Display the full content of the `answer` and `competitors` lists without truncation

print("Answer List:")
pprint(answer)
# Display the answer list as markdown
display(Markdown("### Answer List"))
for idx, ans in enumerate(answer, 1):
    display(Markdown(f"**{idx}.** {ans}"))

# Display the competitors list as markdown
display(Markdown("### Competitors List"))
for idx, comp in enumerate(competitors, 1):
    display(Markdown(f"**{idx}.** {comp}"))
print("\nCompetitors List:")
pprint(competitors)
question = "What are the benefits of using AI Agents?"
interactive_conversation("gpt-4", "gemini-2.0-flash", question, num_turns=1)

Answer List:
['The capital of France is Paris.', 'The capital of France is **Paris**.\n']


### Answer List

**1.** The capital of France is Paris.

**2.** The capital of France is **Paris**.


### Competitors List

**1.** The capital of France is Paris.

**2.** The capital of France is **Paris**.



Competitors List:
['The capital of France is Paris.', 'The capital of France is **Paris**.\n']
Initial Message: What are the benefits of using AI Agents?

Turn 1:
Model 1 (gpt-4) responding...
Competitor Response: 1. Efficiency and Productivity: AI agents can perform tasks faster and more accurately than humans, freeing up human workers to focus on more complex tasks. They can work 24/7 without breaks, improving productivity.

2. Handling Large Volumes of Data: AI agents can process and analyze large amounts of data, which would be impossible for a human to do in a reasonable amount of time. This can lead to better decision-making based on data.

3. Cost Saving: They can replace or reduce the need for human labor in certain tasks, leading to significant cost savings. 

4. Improved Customer Service: AI agents can be used in customer service to respond to customer inquiries quickly and accurately. They can handle multiple inquiries at once, reducing wait times
Model 1 Response: 1. Effic